In [1]:
import numpy as numpy
import pandas as pd

In [2]:
data_path = "D:\VIT\SEM-7\B2 - NLP\Project\Code\curation-corpus-master\output.csv"

In [3]:
df = pd.read_csv(data_path)
df.head()

,title,summary,url,date,article_content
0,Tencent gains approval to sell mutual funds to...,Tencent has been granted a licence from the Ch...,http://www.scmp.com/business/companies/article...,2018-01-04 11:49:58.900000,Traditional finance houses now being seriously...
1,"India testing blockchains in education, health...",India is testing blockchain applications in ed...,https://www.vccircle.com/niti-aayog-explores-b...,2018-01-04 11:33:08.257000,"The Indian government's policy think tank, Nit..."
2,Higher living wage risks robot takeover of low...,The UK Institute for Fiscal Studies has warned...,https://news.sky.com/story/ifs-living-wage-inc...,2018-01-04 11:32:10.223000,Increases in minimum wage levels risk raising ...
3,Regus WeWork may seek stock-market flotation t...,Co-working start-up WeWork may go public this ...,https://www.fool.com/investing/2018/01/03/will...,2018-01-04 11:31:59.617000,"Depending on whom you ask, WeWork is either a ..."
4,AMD poised to gain market share as Intel pound...,Intel has seen more than $11bn wiped off its m...,https://www.cnbc.com/2018/01/05/amd-is-big-win...,2018-01-04 11:17:13.277000,AMD is big winner from chip flaw fiasco as mor...


In [12]:
df_1 = df.drop("date",axis=1)
df_1.dropna(inplace=True)
df_1.head()

,title,summary,url,article_content
0,Tencent gains approval to sell mutual funds to...,Tencent has been granted a licence from the Ch...,http://www.scmp.com/business/companies/article...,Traditional finance houses now being seriously...
1,"India testing blockchains in education, health...",India is testing blockchain applications in ed...,https://www.vccircle.com/niti-aayog-explores-b...,"The Indian government's policy think tank, Nit..."
2,Higher living wage risks robot takeover of low...,The UK Institute for Fiscal Studies has warned...,https://news.sky.com/story/ifs-living-wage-inc...,Increases in minimum wage levels risk raising ...
3,Regus WeWork may seek stock-market flotation t...,Co-working start-up WeWork may go public this ...,https://www.fool.com/investing/2018/01/03/will...,"Depending on whom you ask, WeWork is either a ..."
4,AMD poised to gain market share as Intel pound...,Intel has seen more than $11bn wiped off its m...,https://www.cnbc.com/2018/01/05/amd-is-big-win...,AMD is big winner from chip flaw fiasco as mor...


In [13]:
print("Number of records: ", len(df_1))

Number of records:  37639


In [14]:
print("Null records:\n", df_1.isnull().sum(axis = 0))

Null records:
 title              0
summary            0
url                0
article_content    0
dtype: int64
